# Lab 07-1. Tips

**Import**

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
# for reproducibility
torch.manual_seed(1)

**Training and Test Dataset**

In [4]:
x_train = torch.FloatTensor([[1, 2, 1],
                             [1, 3, 2],
                             [1, 3, 4],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 5],
                             [1, 6, 6],
                             [1, 7, 7]
                            ])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])

In [5]:
x_test = torch.FloatTensor([[2, 1, 1,], [3, 1, 2], [3, 3, 4]])
y_test = torch.LongTensor([2, 2, 2])

**Model**

In [7]:
class SoftmaxClassifierModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 3)
        
    def forward(self, x):
        return self.linear(x)

In [8]:
model = SoftmaxClassifierModel()

In [12]:
# set optimizer
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [27]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 1000
    for epoch in range(nb_epochs + 1):
    
        # H(x) 계산
        prediction = model(x_train)
    
        # cost 계산
        cost = F.cross_entropy(prediction, y_train)
    
        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    
        # 100번마다 로그 출력
        if epoch % 100 == 0:
            print(f'Epoch {epoch:4d}/{nb_epochs}  Cost:{cost.item()}')

In [15]:
def test(model, optimizer, x_test, y_test):
    prediction = model(x_test)
    predicted_classes = prediction.max(1)[1]
    correct_count = (predicted_classes == y_test).sum().item()
    cost = F.cross_entropy(prediction, y_test)
    
    print(f'Accuracy: {correct_count/len(y_test)*100}%  Cost: {cost.item():.6f}')

In [16]:
train(model, optimizer, x_train, y_train)

Epoch    0/1000  Cost:2.2036666870117188
Epoch  100/1000  Cost:0.7206239104270935
Epoch  200/1000  Cost:0.6050705909729004
Epoch  300/1000  Cost:0.5438463091850281
Epoch  400/1000  Cost:0.5014598369598389
Epoch  500/1000  Cost:0.4686412811279297
Epoch  600/1000  Cost:0.4417983889579773
Epoch  700/1000  Cost:0.4191378951072693
Epoch  800/1000  Cost:0.399603933095932
Epoch  900/1000  Cost:0.38250550627708435
Epoch 1000/1000  Cost:0.3673577904701233


In [17]:
test(model, optimizer, x_test, y_test)

Accuracy: 100.0%  Cost: 0.000766


**Learning Rate**

Learnig rate가 너무 크면 diverge하면서 cost가 점점 늘어난다(overshooting).

In [18]:
model = SoftmaxClassifierModel()

In [19]:
optimizer = optim.SGD(model.parameters(), lr=1e5)

In [20]:
train(model, optimizer, x_train, y_train)

Epoch    0/1000  Cost:1.280267596244812
Epoch  100/1000  Cost:896298.75
Epoch  200/1000  Cost:43079.6328125
Epoch  300/1000  Cost:416148.0625
Epoch  400/1000  Cost:679853.875
Epoch  500/1000  Cost:392802.09375
Epoch  600/1000  Cost:0.0
Epoch  700/1000  Cost:0.0
Epoch  800/1000  Cost:0.0
Epoch  900/1000  Cost:0.0
Epoch 1000/1000  Cost:0.0


Learning rate가 너무 작으면 cost가 거의 줄어들지 않는다.

In [21]:
model = SoftmaxClassifierModel()

In [22]:
optimizer = optim.SGD(model.parameters(), lr=1e-10)

In [23]:
train(model, optimizer, x_train, y_train)

Epoch    0/1000  Cost:3.187324047088623
Epoch  100/1000  Cost:3.187324047088623
Epoch  200/1000  Cost:3.187324047088623
Epoch  300/1000  Cost:3.187324047088623
Epoch  400/1000  Cost:3.187324047088623
Epoch  500/1000  Cost:3.187324047088623
Epoch  600/1000  Cost:3.187324047088623
Epoch  700/1000  Cost:3.187324047088623
Epoch  800/1000  Cost:3.187324047088623
Epoch  900/1000  Cost:3.187324047088623
Epoch 1000/1000  Cost:3.187324047088623


적절한 값으로 시작해 발산하면 작게, cost가 줄어들지 않으면 크게 조정하자.

In [24]:
model = SoftmaxClassifierModel()

In [25]:
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [26]:
train(model, optimizer, x_train, y_train)

Epoch    0/1000  Cost:1.3415735960006714
Epoch  100/1000  Cost:0.7052574753761292
Epoch  200/1000  Cost:0.6028958559036255
Epoch  300/1000  Cost:0.5442282557487488
Epoch  400/1000  Cost:0.5023800730705261
Epoch  500/1000  Cost:0.4696085751056671
Epoch  600/1000  Cost:0.4426855146884918
Epoch  700/1000  Cost:0.4199199080467224
Epoch  800/1000  Cost:0.4002859592437744
Epoch  900/1000  Cost:0.3831004500389099
Epoch 1000/1000  Cost:0.36787909269332886


**Data Preprocessing**

In [4]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

**Stantardization**

In [5]:
mu = x_train.mean(dim=0)

In [6]:
sigma = x_train.std(dim=0)

In [7]:
norm_x_train = (x_train - mu) / sigma

In [8]:
print(norm_x_train)

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])


**Training with Preprocessed Data**

In [9]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)
        
    def forward(self, x):
        return self.linear(x)

In [10]:
model = MultivariateLinearRegressionModel()

In [11]:
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [12]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):
    
        # H(x) 계산
        prediction = model(x_train)
    
        # cost 계산
        cost = F.mse_loss(prediction, y_train)
    
        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    
        print(f'Epoch {epoch:4d}/{nb_epochs}  Cost:{cost.item():.6f}')

In [13]:
train(model, optimizer, norm_x_train, y_train)

Epoch    0/20  Cost:29571.447266
Epoch    1/20  Cost:18778.611328
Epoch    2/20  Cost:11975.652344
Epoch    3/20  Cost:7652.159180
Epoch    4/20  Cost:4893.968262
Epoch    5/20  Cost:3131.294678
Epoch    6/20  Cost:2003.926514
Epoch    7/20  Cost:1282.623657
Epoch    8/20  Cost:821.047241
Epoch    9/20  Cost:525.650452
Epoch   10/20  Cost:336.596588
Epoch   11/20  Cost:215.597992
Epoch   12/20  Cost:138.154205
Epoch   13/20  Cost:88.585411
Epoch   14/20  Cost:56.856556
Epoch   15/20  Cost:36.545559
Epoch   16/20  Cost:23.541969
Epoch   17/20  Cost:15.215319
Epoch   18/20  Cost:9.882189
Epoch   19/20  Cost:6.464985
